In [46]:
from sentence_transformers import SentenceTransformer
from PIL import Image
import pytesseract
import fitz
import os
import io
from docx import Document

FOR READING DOCUMENT

In [ ]:
file_path = "test2.docx"

In [ ]:
def extract_text_from_docx(docx_path):
    doc = Document(docx_path)
    collected = []

    # Body paragraphs
    for para in doc.paragraphs:
        text = para.text.strip()
        if text:
            collected.append(text)

    # Tables
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                cell_text = cell.text.strip()
                if cell_text:
                    collected.append(cell_text)

    # Headers & footers
    for section in doc.sections:
        header = section.header
        footer = section.footer

        for para in header.paragraphs:
            if para.text.strip():
                collected.append(para.text.strip())

        for para in footer.paragraphs:
            if para.text.strip():
                collected.append(para.text.strip())

    # Embedded images → OCR
    for rel in doc.part.rels.values():
        if "image" in rel.target_ref:
            image_bytes = rel.target_part.blob
            img = Image.open(io.BytesIO(image_bytes))
            ocr_text = pytesseract.image_to_string(
                img,
                lang="eng",
                config="--psm 6"
            )
            if ocr_text.strip():
                collected.append(ocr_text.strip())

    return "\n".join(collected)

In [56]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    final_text = ""

    for page_num, page in enumerate(doc):
        print(f"Page {page_num + 1}:")

        # Extract digital text
        page_text = page.get_text().strip()
        if page_text:
            print("  Found TEXT")
            final_text += page_text + "\n"

        # OCR only image regions
        images = page.get_images(full=True)
        if images:
            print("  Found IMAGES → OCR image regions")
            for img in images:
                xref = img[0]
                base_image = doc.extract_image(xref)
                image_bytes = base_image["image"]

                img_pil = Image.open(io.BytesIO(image_bytes))
                ocr_text = pytesseract.image_to_string(
                    img_pil,
                    lang="eng",
                    config="--psm 6"
                )

                if ocr_text.strip():
                    final_text += ocr_text + "\n"

        if not page_text and not images:
            print("  Empty page")

    return final_text.strip()


In [50]:
def extract_text_from_image(image_path):
    img = Image.open(image_path)
    return pytesseract.image_to_string(img).strip()

In [58]:
def extract_text(file_path):
    ext = os.path.splitext(file_path)[1].lower()

    if ext == ".pdf":
        print(extract_text_from_pdf(file_path))

    elif ext == ".docx":
        print(extract_text_from_docx(file_path))

    elif ext in [".png", ".jpg", ".jpeg"]:
        print(extract_text_from_image(file_path))

    else:
        raise ValueError("Unsupported file type")
extract_text(file_path)

Beginner Level (6 questions)
Create an unordered list of vegetables
Build a valid HTML page titled "Vegetables" that displays an unordered list (<ul>) with at least 5 items (e.g., Beetroot, Ginger, Potato...).
Add a short description paragraph above the list.
Expected: browser shows the title, paragraph and bulleted list.
CODE -
OUTPUT-
Create an ordered list of steps
Create a page with a numbered (<ol>) list of 5 steps for making tea.
Ensure proper <!DOCTYPE html> and <title>.
Expected: steps appear numbered in order.
CODE-
OUTPUT-
Definition (glossary) list
Build a glossary page using <dl>, <dt>, and <dd> for the terms: HTML (each with a one-line definition).
Expected: terms and definitions shown like a dictionary.
CODE –
OUTPUT-
Simple table with headings
Create a table with a header row (<th>) and two data rows. Columns: Name, Salary. Add border="1".
Expected: header cells bold/centered and two data rows visible.
CODE –
CSS-
OUTPUT-
Formatting text elements
Make a short paragraph t

Chunking document